# SQL

In [25]:
import pandahouse as ph

In [26]:
#объявляем параметры подключения
connection = dict(database='test',
                  host='https://clickhouse.lab.karpov.courses',
                  user='user',
                  password='password')

### Очень усердные ученики
Необходимо написать оптимальный запрос, который даст информацию о количестве очень усердных студентов.

*Под усердным студентом мы понимаем студента, который правильно решил 20 задач за текущий месяц.*
При этом в таблице присутствуют данные только за октябрь, поэтому дату в запросе отображать не будем.

In [27]:
#пишем запрос, и получаем данные из clickhouse в pandas dataframe
query = """
SELECT
    st_id AS st_id,
    SUM(correct) AS correct
FROM peas
GROUP BY
    st_id
HAVING
    correct >= '20'
"""
default_peas = ph.read_clickhouse(query, connection=connection)
print('Количество очень усердных студентов:', len(default_peas))

Количество очень усердных студентов: 136


### Оптимизация воронки

Необходимо в одном запросе выгрузить следующую информацию о группах пользователей:
* ARPU (Average Revenue Per User) — сумма денег, которую приносит каждый пользователь в потоке
* ARPAU (Average Revenue Per Active User) — средний доход на активного пользователя 
* CR в покупку (в процентах)
* СR активного пользователя в покупку (в процентах) 
* CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике (в процентах)

ARPU считается относительно всех пользователей, попавших в группы.

Активным считается пользователь, за все время решивший больше 10 задач правильно в любых дисциплинах.

Активным по математике считается пользователь, за все время решивший 2 или больше задач правильно по математике.


In [72]:
#пишем запрос, и получаем данные из clickhouse в pandas dataframe
query = """
SELECT
    test_grp AS test_group,
    floor(AVG(revenue), 2) AS ARPU,
    floor(AVG(rev_active), 2) AS ARPAU,
    floor(AVG(CASE WHEN revenue > 0 THEN 1 ELSE 0 END), 4)*100 AS CR, -- в процентах
    floor(SUM(revenue > 0 and correct > 10) / SUM(correct > 10), 4)*100 AS CR_active, -- в процентах
    floor(SUM(rev_math = 1) / SUM(correct_math = 1), 4)*100 AS CR_math -- в процентах
FROM
    (
    SELECT
        A.st_id AS st_id,
        A.test_grp AS test_grp,
        B.revenue AS revenue,
        C.correct AS correct,
        CASE WHEN C.correct > 10 THEN B.revenue ELSE 0 END AS rev_active,
        B.rev_math AS rev_math,
        C.correct_math AS correct_math
    FROM studs AS A
    
LEFT JOIN
        (
        SELECT
            st_id,
            SUM(money) AS revenue,
            MAX(CASE WHEN subject = 'Math' THEN 1 ELSE 0 END) AS rev_math
        FROM
            final_project_check 
        GROUP BY
            st_id
        ) AS B 
ON A.st_id = B.st_id

LEFT JOIN
        (
        SELECT
            st_id,
            SUM(correct) AS correct,
            SUM(math) AS correct_math
        FROM
            (
            SELECT
                st_id,
                SUM(correct) AS correct,
                CASE WHEN subject = 'Math' and correct >= 2 THEN 1 ELSE 0 END AS math
            FROM peas
            GROUP BY
                st_id,
                subject
            )
        GROUP BY st_id
        ) AS C
ON A.st_id = C.st_id
    )
GROUP BY test_group
"""
metrics = ph.read_clickhouse(query, connection=connection)
metrics

,test_group,ARPU,ARPAU,CR,CR_active,CR_math
0,control,4540.98,4327.86,4.91,11.02,16.32
1,pilot,11508.47,9677.96,10.84,26.04,38.09
